## Data about vehicle

### Install neccessary packages

In [1]:
!pip install delta-spark==3.2.0

### Import neccessary packages

In [39]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count, hour, when, to_date, round

### Initialize Spark-session with DeltaLake support

In [3]:
spark = SparkSession.builder \
    .appName("TransportDataPipeline") \
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.2.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

### Bronze level - load and save raw data

In [28]:
data = [
    ("2024-01-01 08:30:00", "Toyota", "Corolla", 60, "Kyiv"),
    ("2024-01-01 08:12:00", "Honda", "Civic", 70, "Kyiv"),
    ("2024-01-01 09:30:00", "Mercedes", "Sprinter", 40, "Kyiv"),
    ("2024-01-01 09:40:00", "Ikarus", "256", 45, "Kyiv"),
    ("2024-01-01 09:12:00", "Volvo", "FH16", -10, "Lviv"),
    ("2024-01-01 10:30:00", "Scania", "R500", 220, "Odesa"),
    ("2024-01-01 11:20:00", "Toyota", "Corolla", 60, "Lviv"),
    ("2024-01-01 11:55:00", "Honda", "Civic", 70, "Kyiv"),
    ("2024-01-01 12:30:00", "Mercedes", "Sprinter", 40, "Kyiv"),
    ("2024-01-01 14:30:00", "Ikarus", "256", 45, "Kyiv"),
    ("2024-01-01 18:30:00", "Volvo", "FH16", 80, "Odesa"),
    ("2024-01-01 19:30:00", "Scania", "R500", 85, "Kyiv"),
    ("2024-01-02 08:30:00", "Toyota", "Corolla", 65, "Odesa"),
    ("2024-01-02 09:30:00", "Honda", "Civic", 75, "Lviv"),
    ("2024-01-02 09:20:00", "Mercedes", "Sprinter", 50, "Lviv"),
    ("2024-01-02 09:15:00", "Ikarus", "256", 55, "Kyiv"),
    ("2024-01-02 10:30:00", "Volvo", "FH16", 90, "Lviv"),
    ("2024-01-02 10:55:00", "Scania", "R500", 95, "Kyiv"),
    ("2024-01-02 11:30:00", "Toyota", "Corolla", 220, "Lviv"),
    ("2024-01-02 12:30:00", "Honda", "Civic", 75, "Lviv"),
    ("2024-01-02 18:30:00", "Mercedes", "Sprinter", 50, "Odesa"),
    ("2024-01-02 19:30:00", "Ikarus", "256", 55, "Lviv"),
    ("2024-01-02 19:40:00", "Volvo", "FH16", 90, "Kyiv"),
    ("2024-01-02 19:50:00", "Scania", "R500", 95, "Lviv"),
    ("2024-01-03 08:30:00", "Toyota", "Corolla", 70, "Kyiv"),
    ("2024-01-03 09:30:00", "Honda", "Civic", 80, "Odesa"),
    ("2024-01-03 09:40:00", "Mercedes", "Sprinter", 55, "Odesa"),
    ("2024-01-03 09:45:00", "Ikarus", "256", 60, "Lviv"),
    ("2024-01-03 10:30:00", "Volvo", "FH16", 100, "Lviv"),
    ("2024-01-03 12:30:00", "Scania", "R500", 110, "Odesa"),
    ("2024-01-03 14:30:00", "Toyota", "Corolla", -10, "Kyiv"),
    ("2024-01-03 15:30:00", "Honda", "Civic", 80, "Odesa"),
    ("2024-01-03 16:30:00", "Mercedes", "Sprinter", 55, "Odesa"),
    ("2024-01-03 19:30:00", "Ikarus", "256", 60, "Odesa"),
    ("2024-01-03 20:30:00", "Volvo", "FH16", 250, "Odesa"),
    ("2024-01-03 21:30:00", "Scania", "R500", 110, "Kyiv")
]

columns = ["timestamp", "brand", "model", "speed", "city"]
bronze_df = spark.createDataFrame(data, columns)
bronze_df.write.format("delta").mode("overwrite").save("/home/jovyan/work/lab14_data/bronze")

### Check data

In [29]:
bronze_df.show()

+-------------------+--------+--------+-----+-----+
|          timestamp|   brand|   model|speed| city|
+-------------------+--------+--------+-----+-----+
|2024-01-01 08:30:00|  Toyota| Corolla|   60| Kyiv|
|2024-01-01 08:12:00|   Honda|   Civic|   70| Kyiv|
|2024-01-01 09:30:00|Mercedes|Sprinter|   40| Kyiv|
|2024-01-01 09:40:00|  Ikarus|     256|   45| Kyiv|
|2024-01-01 09:12:00|   Volvo|    FH16|  -10| Lviv|
|2024-01-01 10:30:00|  Scania|    R500|  220|Odesa|
|2024-01-01 11:20:00|  Toyota| Corolla|   60| Lviv|
|2024-01-01 11:55:00|   Honda|   Civic|   70| Kyiv|
|2024-01-01 12:30:00|Mercedes|Sprinter|   40| Kyiv|
|2024-01-01 14:30:00|  Ikarus|     256|   45| Kyiv|
|2024-01-01 18:30:00|   Volvo|    FH16|   80|Odesa|
|2024-01-01 19:30:00|  Scania|    R500|   85| Kyiv|
|2024-01-02 08:30:00|  Toyota| Corolla|   65|Odesa|
|2024-01-02 09:30:00|   Honda|   Civic|   75| Lviv|
|2024-01-02 09:20:00|Mercedes|Sprinter|   50| Lviv|
|2024-01-02 09:15:00|  Ikarus|     256|   55| Kyiv|
|2024-01-02 

### Silver level - clear and categorize data

In [36]:
bronze_data = spark.read.format("delta").load("/home/jovyan/work/lab14_data/bronze")
silver_df = bronze_data.dropDuplicates()
silver_df = silver_df.filter((col("speed") >= 0) & (col("speed") <= 200))

# Додавання категоризації транспорту
silver_df = silver_df.withColumn(
    "vehicle_type", 
    when(col("brand").isin(["Toyota", "Honda"]), "personal")
    .when(col("brand").isin(["Mercedes", "Ikarus"]), "public")
    .when(col("brand").isin(["Volvo", "Scania"]), "cargo")
    .otherwise("unknown")
)

# Додавання окремих колонок для часу та дати
silver_df = silver_df.withColumn("date", to_date(col("timestamp")))
silver_df = silver_df.withColumn("hour", hour(col("timestamp")))

silver_df.write.format("delta").mode("overwrite").save("/home/jovyan/work/lab14_data/silver")

### Check data

In [37]:
silver_df.show()

+-------------------+--------+--------+-----+-----+------------+----------+----+
|          timestamp|   brand|   model|speed| city|vehicle_type|      date|hour|
+-------------------+--------+--------+-----+-----+------------+----------+----+
|2024-01-02 08:30:00|  Toyota| Corolla|   65|Odesa|    personal|2024-01-02|   8|
|2024-01-02 09:20:00|Mercedes|Sprinter|   50| Lviv|      public|2024-01-02|   9|
|2024-01-02 10:30:00|   Volvo|    FH16|   90| Lviv|       cargo|2024-01-02|  10|
|2024-01-02 10:55:00|  Scania|    R500|   95| Kyiv|       cargo|2024-01-02|  10|
|2024-01-02 09:15:00|  Ikarus|     256|   55| Kyiv|      public|2024-01-02|   9|
|2024-01-02 09:30:00|   Honda|   Civic|   75| Lviv|    personal|2024-01-02|   9|
|2024-01-03 21:30:00|  Scania|    R500|  110| Kyiv|       cargo|2024-01-03|  21|
|2024-01-03 16:30:00|Mercedes|Sprinter|   55|Odesa|      public|2024-01-03|  16|
|2024-01-03 19:30:00|  Ikarus|     256|   60|Odesa|      public|2024-01-03|  19|
|2024-01-03 15:30:00|   Hond

### Gold level - clear and categorize data

In [44]:
silver_data = spark.read.format("delta").load("/home/jovyan/work/lab14_data/silver")

# Агрегування по місту та даті
gold_city_date_agg = silver_data.groupBy("city", "date").agg(
    round(avg("speed"), 2).alias("average_speed"),
    count("brand").alias("vehicle_count")
).orderBy("date")

# Агрегування по місту та годину
gold_peak_hours = silver_data.groupBy("city", "hour").agg(
    count("brand").alias("vehicle_count")
).orderBy("hour")

# Зберігаємо звіти в окремих бд, щоб пізніше провести аналіз
gold_city_date_agg.write.format("delta").mode("overwrite").save("/home/jovyan/work/lab14_data/gold_city_date_agg")
gold_peak_hours.write.format("delta").mode("overwrite").save("/home/jovyan/work/lab14_data/gold_peak_hours")

### Show data for analyze

In [47]:
# Завантажуємо потрібні дані (звіти) для аналізу
gold_city_date_agg_data = spark.read.format("delta").load("/home/jovyan/work/lab14_data/gold_city_date_agg")
gold_peak_hours_data = spark.read.format("delta").load("/home/jovyan/work/lab14_data/gold_peak_hours")

# Виводимо дані для аналізу середньої швидкості та кількості транспорту в певні дати та місту
gold_city_date_agg.show()

# Виводимо дані для аналізу часу-пік в різних містах
gold_peak_hours.show()

# Можна зробити висновок, що час-пік в місті Києві - це 8-9 години, у Львові - 9 година, а в Одесі трафік рівномірний протягом дня

+-----+----------+-------------+-------------+
| city|      date|average_speed|vehicle_count|
+-----+----------+-------------+-------------+
| Lviv|2024-01-01|         60.0|            1|
| Kyiv|2024-01-01|        56.88|            8|
|Odesa|2024-01-01|         80.0|            1|
| Lviv|2024-01-02|        73.33|            6|
| Kyiv|2024-01-02|         80.0|            3|
|Odesa|2024-01-02|         57.5|            2|
|Odesa|2024-01-03|        73.33|            6|
| Lviv|2024-01-03|         80.0|            2|
| Kyiv|2024-01-03|         90.0|            2|
+-----+----------+-------------+-------------+

+-----+----+-------------+
| city|hour|vehicle_count|
+-----+----+-------------+
|Odesa|   8|            1|
| Kyiv|   8|            3|
| Lviv|   9|            3|
| Kyiv|   9|            3|
|Odesa|   9|            2|
| Lviv|  10|            2|
| Kyiv|  10|            1|
| Kyiv|  11|            1|
| Lviv|  11|            1|
| Lviv|  12|            1|
|Odesa|  12|            1|
| Kyiv|  1

In [ ]:
spark.stop()